# **Exploring Mental Health - Playground Series PS4S11**


# **Setup**

In [1]:
%%capture

# AutoGluon is a SOTA AutoML framework
%pip install -q setuptools wheel autogluon.tabular[all,skex] dask[dataframe]
%pip install -U -q ipywidgets cloudpickle==2.2.1

In [2]:
# %%capture

# # LightGBM GPU instalation (restart after installing)
# ! git clone --recursive https://github.com/Microsoft/LightGBM

# #You can run this oneliner which will build and compile LightGBM with GPU enabled in colab:
# ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
# !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os
import cloudpickle
import re
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
base_path = os.getenv('DATA_FOLDER_PATH', '/content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/')

train_data = TabularDataset(os.path.join(base_path, 'train_cleaned.csv'))
test_data = TabularDataset(os.path.join(base_path, 'test_cleaned.csv'))
submission = TabularDataset(os.path.join(base_path, 'sample_submission.csv'))

In [6]:
train_data.drop(columns=['is_original', 'cgpa_bucket'], inplace=True)

# **Autogluon 8hr Training**

In [ ]:
# Setting up
eval_metric = 'log_loss'
label = 'depression'
problem_type='binary'
hours = 8

# Models to exclude
excluded_model_types = ['KNN']

# Initialize the TabularPredictor
predictor = TabularPredictor(label=label, eval_metric=eval_metric, problem_type=problem_type,
                             path = os.path.join(base_path, "Autogluon/202411_ps4s11_8hr_logloss_gpu"))

# Fit the model
predictor.fit(train_data=train_data,
              time_limit=3600*hours,
              presets="best_quality",
              excluded_model_types=excluded_model_types,
              num_bag_folds=10,
              num_bag_sets = 10,
              num_stack_levels=3,
              full_weighted_ensemble_additionally=True,
              ag_args_fit={'num_gpus': 1}
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          8
Memory Avail:       48.77 GB / 50.99 GB (95.6%)
Disk Space Avail:   184.02 GB / 235.68 GB (78.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=10, num_bag_sets=10
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect st

In [ ]:
predictor = TabularPredictor.load(os.path.join(base_path, "Autogluon/202411_ps4s11_8hr_logloss_gpu"))

In [ ]:
leaderboard_test = predictor.leaderboard()
leaderboard_test.head(10)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_ALL_L5,-0.148255,log_loss,105.585109,10171.116691,0.003628,1.185391,5,True,87
1,WeightedEnsemble_L2,-0.148289,log_loss,46.990007,4071.147295,0.003508,1.193801,2,True,28
2,WeightedEnsemble_L3,-0.148385,log_loss,126.403947,11790.050694,0.003290,1.067314,3,True,52
3,WeightedEnsemble_L4,-0.148872,log_loss,164.148464,17469.321232,0.005054,0.978731,4,True,72
4,XGBoost_BAG_L2,-0.148991,log_loss,99.243326,7134.348668,0.584466,41.935683,2,True,37
5,CatBoost_BAG_L2,-0.149090,log_loss,98.804893,7189.119752,0.146032,96.706767,2,True,33
6,LightGBM_r96_BAG_L2,-0.149216,log_loss,102.429577,7197.745426,3.770716,105.332442,2,True,45
7,CatBoost_r137_BAG_L2,-0.149249,log_loss,98.787518,7154.330236,0.128657,61.917252,2,True,49
8,LightGBM_r131_BAG_L2,-0.149251,log_loss,102.657257,7234.507014,3.998396,142.094029,2,True,42
9,NeuralNetTorch_r79_BAG_L2,-0.149260,log_loss,100.322798,7628.715830,1.663937,536.302845,2,True,41


## **Submission**

In [ ]:
# Calibrate the prediction threshold
predictor.calibrate_decision_threshold(train_data, metric='accuracy')

0.511

In [ ]:
model = "WeightedEnsemble_ALL_L5"
suffix = "m1"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9401072206399732


In [ ]:
model = "WeightedEnsemble_L2"
suffix = "m2"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9402398503378567


In [ ]:
model = "WeightedEnsemble_L3"
suffix = "m3"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9401351426816329


In [ ]:
model = "WeightedEnsemble_L4"
suffix = "m4"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.940302674931591


In [ ]:
model = "XGBoost_BAG_L2"
suffix = "m5"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9401351426816329


In [ ]:
model = "CatBoost_BAG_L2"
suffix = "m6"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.940211928296197


# **Autogluon 20hr training Accuracy**

In [ ]:
# Setting up
eval_metric = 'accuracy'
label = 'depression'
problem_type='binary'
hours = 20

# Models to exclude
excluded_model_types = ['KNN']

# Initialize the TabularPredictor
predictor = TabularPredictor(label=label, eval_metric=eval_metric, problem_type=problem_type,
                             path = os.path.join(base_path, "Autogluon/202411_ps4s11_20hr_accuracy"))

# Fit the model
predictor.fit(train_data=train_data,
              time_limit=3600*hours,
              presets="best_quality",
              excluded_model_types=excluded_model_types,
              hyperparameter_tune_kwargs="auto",
              num_bag_folds=10,
              num_bag_sets = 10,
              num_stack_levels=3,
              full_weighted_ensemble_additionally=True
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          8
Memory Avail:       48.57 GB / 50.99 GB (95.2%)
Disk Space Avail:   174.95 GB / 225.83 GB (77.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=10, num_bag_sets=10
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect st

(_dystack pid=4732) ╭───────────────────────────────────────────────────────────╮
(_dystack pid=4732) │ Configuration for experiment     NeuralNetFastAI_BAG_L1   │
(_dystack pid=4732) ├───────────────────────────────────────────────────────────┤
(_dystack pid=4732) │ Search algorithm                 SearchGenerator          │
(_dystack pid=4732) │ Scheduler                        FIFOScheduler            │
(_dystack pid=4732) │ Number of trials                 1000                     │
(_dystack pid=4732) ╰───────────────────────────────────────────────────────────╯
(_dystack pid=4732) 
(_dystack pid=4732) View detailed results here: /content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/Autogluon/202411_ps4s11_20hr_accuracy/ds_sub_fit/sub_fit_ho/models/NeuralNetFastAI_BAG_L1


(_dystack pid=4732) Reached timeout of 49.97602127337307 seconds. Stopping all trials.
(_dystack pid=4732) Wrote the latest version of all result files and experiment state to '/content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/Autogluon/202411_ps4s11_20hr_accuracy/ds_sub_fit/sub_fit_ho/models/NeuralNetFastAI_BAG_L1' in 0.0275s.
(_dystack pid=4732) Failed to fetch metrics for 4 trial(s):
(_dystack pid=4732) - 3cecdc0e: FileNotFoundError('Could not fetch metrics for 3cecdc0e: both result.json and progress.csv were not found at /content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/Autogluon/202411_ps4s11_20hr_accuracy/ds_sub_fit/sub_fit_ho/models/NeuralNetFastAI_BAG_L1/3cecdc0e')
(_dystack pid=4732) - bf358476: FileNotFoundError('Could not fetch metrics for bf358476: both result.json and progress.csv were not found at /content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/

(_dystack pid=4732) 


(_dystack pid=4732) Hyperparameter tuning model: XGBoost_BAG_L1 ... Tuning model for up to 49.98s of the 17770.54s of remaining time.
(_dystack pid=4732) 	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.18%)
(_dystack pid=4732) 	Stopping HPO to satisfy time limit...
(_dystack pid=4732) Fitted model: XGBoost_BAG_L1/T1 ...
(_dystack pid=4732) 	0.9397	 = Validation score   (accuracy)
(_dystack pid=4732) 	36.96s	 = Training   runtime
(_dystack pid=4732) 	0.93s	 = Validation runtime
(_dystack pid=4732) Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ... Tuning model for up to 49.98s of the 17733.45s of remaining time.
(_dystack pid=4732) [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


...

+-------------------------------------------------------------+
| Configuration for experiment     NeuralNetTorch_r1_BAG_L2   |
+-------------------------------------------------------------+
| Search algorithm                 SearchGenerator            |
| Scheduler                        FIFOScheduler              |
| Number of trials                 1000                       |
+-------------------------------------------------------------+

View detailed results here: /content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/Autogluon/202411_ps4s11_20hr_accuracy/models/NeuralNetTorch_r1_BAG_L2


2024-11-14 13:21:38,703	INFO timeout.py:54 -- Reached timeout of 153.2251231050319 seconds. Stopping all trials.
2024-11-14 13:21:38,727	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-11-14 13:21:38,764	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/Autogluon/202411_ps4s11_20hr_accuracy/models/NeuralNetTorch_r1_BAG_L2' in 0.0462s.
2024-11

2024-11-14 13:21:42,747	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+--------------------------------------------------------------+
| Configuration for experiment     NeuralNetTorch_r89_BAG_L2   |
+--------------------------------------------------------------+
| Search algorithm                 SearchGenerator             |
| Scheduler                        FIFOScheduler               |
| Number of trials                 1000                        |
+--------------------------------------------------------------+

View detailed results here: /content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/Autogluon/202411_ps4s11_20hr_accuracy/models/NeuralNetTorch_r89_BAG_L2


2024-11-14 13:24:16,139	INFO timeout.py:54 -- Reached timeout of 153.2251231050319 seconds. Stopping all trials.
2024-11-14 13:24:16,150	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-11-14 13:24:16,181	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/content/drive/MyDrive/DS_Projects/Playground_Series/Ps4e11_Exploring_Mental_Health_Data/Data/Autogluon/202411_ps4s11_20hr_accuracy/models/NeuralNetTorch_r89_BAG_L2' in 0.0383s.
2024-1

In [7]:
predictor = TabularPredictor.load(os.path.join(base_path, "Autogluon/202411_ps4s11_20hr_accuracy"))

In [8]:
leaderboard_test = predictor.leaderboard()
leaderboard_test.head(10)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge_BAG_L4,0.941517,accuracy,961.737769,23596.064260,0.818142,42.878393,4,True,360
1,WeightedEnsemble_L5,0.941517,accuracy,961.746224,23601.403514,0.008455,5.339254,5,True,429
2,WeightedEnsemble_ALL_L5,0.941517,accuracy,961.752530,23601.227719,0.014761,5.163459,5,True,428
3,XGBoost_BAG_L4/T1,0.941434,accuracy,962.307966,23597.535761,1.388339,44.349894,4,True,358
4,LightGBM_BAG_L4/T1,0.941273,accuracy,961.320187,23585.443075,0.400560,32.257209,4,True,353
5,CatBoost_r70_BAG_L4,0.941238,accuracy,961.067057,23591.278347,0.147430,38.092480,4,True,381
6,LightGBM_BAG_L3/T3,0.941231,accuracy,629.629249,16874.011506,0.542470,33.741126,3,True,251
7,WeightedEnsemble_L4,0.941231,accuracy,629.643276,16879.541226,0.014027,5.529721,4,True,350
8,CatBoost_r180_BAG_L3,0.941210,accuracy,629.337719,16893.508550,0.250940,53.238170,3,True,330
9,CatBoost_r9_BAG_L4,0.941210,accuracy,961.250075,23597.961420,0.330448,44.775553,4,True,364


## **Submission**

In [9]:
# Calibrate the prediction threshold
predictor.calibrate_decision_threshold(train_data, metric='accuracy')

0.5690000000000001

In [16]:
model = "LightGBMLarge_BAG_L4"
suffix = "m1"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.57, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon2_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon2_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9415172837437873


In [10]:
model = "LightGBMLarge_BAG_L4"
suffix = "m1"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.50, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon2_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon2_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9415172837437873


In [11]:
model = "WeightedEnsemble_L5"
suffix = "m2"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon2_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon2_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9415172837437873


In [12]:
model = "WeightedEnsemble_ALL_L5"
suffix = "m3"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon2_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon2_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9415172837437873


In [13]:
model = "XGBoost_BAG_L4/T1"
suffix = "m4"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon2_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon2_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9414335176188083


In [14]:
model = "LightGBM_BAG_L4/T1"
suffix = "m5"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon2_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon2_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9412729658792651


In [15]:
model = "CatBoost_r70_BAG_L4"
suffix = "m6"
sub_autogluon = submission.copy()
sub_autogluon['Depression'] = predictor.predict_proba(test_data, as_pandas=False, model=model)[:, 1]
sub_autogluon['Depression'] = np.where(sub_autogluon['Depression'] > 0.5, 1, 0)
sub_autogluon.to_csv(os.path.join(base_path, f'submission_autogluon2_{suffix}.csv'), index=False)

# Accuracy
oof_autogluon = predictor.predict_proba_oof(model=model)[1]
oof_autogluon.to_csv(os.path.join(base_path, f'oof_autogluon2_{suffix}.csv'), index=False)
predicted_classes = (oof_autogluon > 0.50).astype(int)
accuracy = accuracy_score(train_data['depression'], predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9412380633271905
